In [14]:
%matplotlib inline
import pylab as pl
import numpy as np
import pandas as pd

#Read data

X = []
samples_x = []

f = open("genotypes.csv","r")

for i,line in enumerate(f):
    if(i == 0):
       continue
    
    sv = line.strip().split(",")
    samples_x.append(sv[0])
    
    row = []
    for element in sv[1:]:
        row.append(element)
    X.append(row)
    
f.close()

X = np.array(X)
samples_x = np.array(samples_x)

print(samples_x)
print(X)


['9597' '9143' '8967' '5577' '7127' '9309' '9929' '5901' '9636' '2053'
 '8259' '5678' '1651' '9895' '9411' '280' '7460' '6983' '9965' '6942'
 '9782' '1740' '9131' '264' '9870' '9530' '6145' '5932' '6141' '391'
 '9880' '6149' '9384' '198' '5466' '1552' '185' '8996' '2212' '8213'
 '4758' '7014' '278' '8264' '9957' '4' '9670' '6106' '6906' '6020' '8701'
 '9876' '9731' '5606' '9725' '8725' '6236' '7102' '9819' '367' '9976'
 '9822' '1890' '8787' '2' '8395' '9952' '9809' '9700' '19950' '7379'
 '7506' '2412' '719' '9963' '5207' '5902' '9369' '5899' '9832' '6129'
 '1362' '9796' '8727' '200' '373' '9894' '6974' '5373' '8304' '9559' '122'
 '6116' '6151' '7000' '6847' '201' '8300' '1733' '7177' '6103' '6969'
 '184' '9508' '5994' '214' '9637' '377' '9653' '8973' '6909' '5890'
 '10005' '329' '7301' '5385' '205' '7268' '6981' '9611' '6128' '2171'
 '6096' '7008' '5872' '263' '6095' ... '6207' '6134' '9714' '5914' '4884'
 '8357' '10014' '5893' '178' '369' '5993' '9673' '362' '7378' '6105'
 '9152' '719

In [15]:
#Replace "?" with np.nan

X = np.where(X == "?", np.nan, X)

print(X)

[['G' 'G' 'G' 'C' 'G' 'C' 'C' 'T' 'G' 'C' 'A' 'C' 'nan' 'A' 'G' 'T' 'A'
  'nan' 'A' 'T' 'G' 'T' 'C' 'T' 'T' 'A' 'nan' 'T' 'C' 'C' 'G' 'T' 'G' 'A'
  'G' 'A' 'nan' 'T' 'C' 'C' 'T' 'A' 'T' 'T' 'A' 'A' 'T' 'G' 'G' 'C' 'T'
  'A' 'nan' 'G' 'A' 'A' 'C' 'G' 'C' 'T' 'A' 'G' 'T' 'G' 'C' 'A' 'G' 'T'
  'A' 'T' 'A' 'G' 'T' 'G' 'T' 'A' 'G' 'C' 'C' 'C' 'T' 'A' 'nan' 'C' 'nan'
  'A' 'T' 'C' 'T' 'C' 'G' 'nan' 'G' 'T' 'G' 'G' 'nan' 'A' 'A' 'A' 'G' 'T'
  'T' 'nan' 'A' 'C' 'T' 'A' 'T' 'C' 'A' 'T' 'T' 'A' 'nan' 'C' 'A' 'G' 'C'
  'G' 'T' 'G' 'A' 'A' 'T' 'C' 'G' ... 'C' 'nan' 'G' 'T' 'nan' 'A' 'C' 'A'
  'T' 'T' 'G' 'A' 'T' 'C' 'G' 'T' 'T' 'T' 'C' 'G' 'T' 'G' 'A' 'C' 'nan'
  'T' 'A' 'C' 'C' 'C' 'C' 'G' 'G' 'nan' 'G' 'nan' 'C' 'T' 'nan' 'nan' 'C'
  'T' 'A' 'G' 'T' 'A' 'C' 'C' 'T' 'nan' 'T' 'nan' 'T' 'T' 'C' 'C' 'nan'
  'A' 'T' 'G' 'A' 'G' 'G' 'C' 'A' 'C' 'G' 'C' 'G' 'T' 'C' 'nan' 'A' 'C'
  'nan' 'A' 'C' 'G' 'T' 'T' 'G' 'A' 'C' 'A' 'C' 'C' 'C' 'T' 'T' 'C' 'nan'
  'T' 'G' 'G' 'G' 'T' 'C' 'G' 'G' 'nan' 'C' 'T' 'A

In [16]:
#Encode nucleotide values

for j in range (X.shape[1]):
    
    values, counts = np.unique(X[:,j], return_counts=True)
    values=values[:2]
    counts=counts[:2]

    #print(values[np.argmax(counts)])
    #print(values[np.argmin(counts)])

    X[:,j] = np.where(X[:,j] == values[np.argmax(counts)], 0.0, X[:,j])
    X[:,j] = np.where(X[:,j] == values[np.argmin(counts)], 2.0, X[:,j])

#Convert array from string to float
X = X.astype(np.float_)

print(X)
    

[[ 0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.  0. nan  0.  0.  0.  0. nan
   0.  0.  0.  0.  0.  2.  2.  2. nan  0.  0.  0.  0.  2.  0.  0.  0.  0.
  nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  2. nan  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  2.  0.  0.  0.
   2.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0. nan  0.  0.  0.  0.  2.
   0. nan  0.  0.  0.  0. nan  0.  0.  0.  0.  0.  0. nan  2.  0.  0.  0.
   0.  0.  0.  2.  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0. ...  0. nan  0.  0. nan  0.  0.  2.  0.  0.  0.  0.  2.  0.  0.  2.
   0.  0.  0.  0.  0.  0.  0.  0. nan  0.  0.  0.  2.  0.  0.  0.  0. nan
   0. nan  0.  0. nan nan  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0. nan
   0.  0.  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0. nan  0.  0. nan  0.  0.  0.  0.  2.  0.  0.  2.  0.  0.  0.  0.  0.
   0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0. nan  0.  2.  0.  0.  0.  0.
   0.  0. nan  0.  0.  0.  0.  0.  0. 

In [17]:
# General analysis data:

print("No Samples:\t\t %d" % X.shape[0])
print("No Features:\t\t %d" % X.shape[1])

missing = ((np.isnan(X)).sum()/X.size*100)
print("Missing Values:\t\t %.2f%%" % missing)

No Samples:		 1826
No Features:		 5000
Missing Values:		 10.00%


In [18]:
# Assess missing values per sample


sample_indices_to_remove = []
for i in range(X.shape[0]):
    missing = np.isnan(X[i,:]).sum()/X.shape[1]*100
    if missing>50:
        print("Sample %d has %.2f missing values" % (i,missing))
        sample_indices_to_remove.append(i)

if (len(sample_indices_to_remove) == 0):
    print("All samples have less than 50 % missing values")

All samples have less than 50 % missing values


In [19]:
# Read phenotype values

y = []
samples_y = []

f = open("phenotype_values.csv","r")

for i,line in enumerate(f):
    if(i == 0):
       continue
    sv = line.strip().split(",")
    samples_y.append(sv[0])
    y.append(float(sv[1]))
    
f.close()

y = np.array(y)
samples_y = np.array(samples_y)

print (samples_y)
print (y)

['9608' '9801' '9647' '7014' '9802' '7523' '9689' '6177' '7340' '8534'
 '15592' '9850' '7096' '7207' '5232' '9946' '9857' '8386' '9657' '9344'
 '7202' '6417' '628' '9153' '6235' '6098' '7268' '2019' '868' '9070'
 '9114' '10006' '6910' '5350' '266' '7026' '9646' '1720' '87' '6091'
 '1585' '5994' '2' '9820' '6149' '5805' '9001' '2157' '5466' '6904' '9598'
 '9677' '2290' '7526' '223' '410' '9131' '323' '2286' '9784' '10001'
 '7028' '2191' '2011' '5678' '7917' '351' '5968' '1256' '6974' '1782'
 '5835' '6913' '1942' '5381' '9858' '7392' '607' '8464' '9527' '1064'
 '6220' '6153' '215' '8266' '2274' '8411' '207' '2187' '6973' '9837'
 '9910' '9908' '9895' '2180' '9928' '5353' '1851' '9986' '7120' '1954'
 '309' '6085' '915' '9768' '8957' '7075' '4935' '615' '14318' '6141'
 '5712' '1435' '5915' '8774' '7354' '1153' '9835' '4884' '7252' '293'
 '10023' '5784' '9871' '5901' '9653' '6905' ... '6268' '7244' '6961'
 '9878' '9762' '6994' '9630' '9378' '7133' '7011' '9540' '9405' '9822'
 '6897' '8430' '

In [20]:
# Match genotypes with phenotype values
truth_table = (samples_x[:,np.newaxis]==samples_y)
ind = np.where(truth_table==True)

samples_x = samples_x[ind[0]]
samples_y = samples_y[ind[1]]

X = X[ind[0],:]
y = y[ind[1]]

print("No Samples X:\t\t%d" % X.shape[0])
print("No Samples y:\t\t%d" % y.shape[0])

print("Samples correct order:\t" + str(np.any(samples_x==samples_y)))


No Samples X:		1826
No Samples y:		1826
Samples correct order:	True


In [21]:
# Split into train data and test data

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=73)

print("Full Data:\t" + str(X.shape))
print("Train Data:\t" + str(X_train.shape))
print("Test Data:\t" + str(X_test.shape))



Full Data:	(1826, 5000)
Train Data:	(1460, 5000)
Test Data:	(366, 5000)


In [22]:
# Impute missing values on training data

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="most_frequent")
X_train = imputer.fit_transform(X_train) 
X_test = imputer.fit_transform(X_test) 

print(X_train)


[[0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 2. 0. 0. 2. 0. 2. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 2. 0.
  2. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0.
  0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 2. 0. ... 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2.
  0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 2. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 2. 0. 0. 2. 0. 0. 0. 0. 0. 0. 2. 0. 2. 0. 2. 0. 0. 0. 0. 0. 0.
  2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 2. 0. 0. 0. 0. 2. 0. 0.
  0. 2. 0. 0. 0. 2. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [23]:
# Linear Regression

import sklearn.metrics as metrics
from sklearn import linear_model


model1 = linear_model.LinearRegression()
model1.fit(X_train, y_train)
predictions_training = model1.predict(X_train)
predictions_testing = model1.predict(X_test)

print("MSE (Train):\t%.2f" % metrics.mean_squared_error(y_train, predictions_training))
print("R2 (Train):\t%.2f" % metrics.r2_score(y_train, predictions_training))
print("MSE (Test):\t%.2f" % metrics.mean_squared_error(y_test, predictions_testing))
print("R2 (Test):\t%.2f" % metrics.r2_score(y_test, predictions_testing))

MSE (Train):	0.00
R2 (Train):	1.00
MSE (Test):	4.50
R2 (Test):	0.55


In [24]:
from sklearn import linear_model
model2=linear_model.Ridge(alpha=5000)
model2.fit(X_train, y_train)
predictions_training = model2.predict(X_train)
predictions_testing = model2.predict(X_test)

print("MSE (Train):\t%.2f" % metrics.mean_squared_error(y_train, predictions_training))
print("R2 (Train):\t%.2f" % metrics.r2_score(y_train, predictions_training))
print("MSE (Test):\t%.2f" % metrics.mean_squared_error(y_test, predictions_testing))
print("R2 (Test):\t%.2f" % metrics.r2_score(y_test, predictions_testing))

MSE (Train):	2.79
R2 (Train):	0.73
MSE (Test):	4.63
R2 (Test):	0.54


In [25]:
from sklearn import linear_model
model3=linear_model.ElasticNet(alpha=0.1,l1_ratio=0.1,max_iter=7000)
model3.fit(X_train, y_train)
predictions_training = model3.predict(X_train)
predictions_testing = model3.predict(X_test)

print("MSE (Train):\t%.2f" % metrics.mean_squared_error(y_train, predictions_training))
print("R2 (Train):\t%.2f" % metrics.r2_score(y_train, predictions_training))
print("MSE (Test):\t%.2f" % metrics.mean_squared_error(y_test, predictions_testing))
print("R2 (Test):\t%.2f" % metrics.r2_score(y_test, predictions_testing))

MSE (Train):	0.50
R2 (Train):	0.95
MSE (Test):	3.79
R2 (Test):	0.62


c:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.809e+00, tolerance: 1.519e+00
  model = cd_fast.enet_coordinate_descent(


In [26]:
np.set_printoptions(edgeitems=127)
#print(model1.intercept_,model1.coef_)
#print(model2.intercept_,model2.coef_)
print(model3.intercept_,model3.coef_.size,model3.coef_)

51.82642688820207 5000 [ 0.         -0.02389207  0.0046859  -0.          0.04037196 -0.
  0.03030297  0.03483873  0.         -0.0111967  -0.         -0.
  0.          0.          0.         -0.         -0.         -0.
  0.          0.         -0.          0.         -0.         -0.00151823
 -0.          0.02235127 -0.          0.01299651 -0.00514956  0.01612248
 -0.04460945  0.01759275 -0.          0.02220632  0.         -0.
  0.00631059  0.00257745 -0.          0.         -0.         -0.
  0.04408177  0.09451984  0.          0.11159349 -0.         -0.
 -0.          0.          0.          0.01937423 -0.06442915  0.00816678
 -0.         -0.         -0.          0.          0.         -0.00884932
  0.         -0.         -0.         -0.05592441 -0.         -0.00785153
 -0.03061023  0.         -0.         -0.          0.01746757 -0.
 -0.02559057 -0.06922651 -0.         -0.         -0.02386255  0.06213533
  0.          0.04536358 -0.         -0.         -0.         -0.
 -0.         -0.   